In [50]:
import pandas as pd
import numpy as np
import keras
from keras import layers
from keras import callbacks
from loguru import logger
from sklearn.model_selection import train_test_split

In [43]:
TEST_SAMPLE_FILE_PATH, TRAIN_SAMPLE_FILE_PATH = 'data/fashion-mnist_test.csv', 'data/fashion-mnist_train.csv'

test_sample_df = pd.read_csv(TEST_SAMPLE_FILE_PATH)
train_sample_df = pd.read_csv(TRAIN_SAMPLE_FILE_PATH)


# Подготовка данных

Разделим данные обучения и тестирования на массивы признаков и массивы меток. Признаки будем нормализовать, разделив на 255. Для обучения модели разделим данные обучения на обучающую и валидационную выборки в соотношении 80/20.

In [46]:

train_data = np.array(train_sample_df, dtype='float32')
test_data = np.array(test_sample_df, dtype='float32')

x_train = train_data[:, 1:] / 255
y_train = train_data[:, 0]

x_test = test_data[:, 1:] / 255
y_test = test_data[:, 0]

In [48]:
x_train, x_validate, y_train, y_validate = train_test_split(x_train, y_train, test_size=0.2, random_state=228)

# Настройки модели и запуск

Для создания модели будем использовать следующие настройки:
- `ImgSettings` - настройки изображения
- `LayersSettings` - настройки слоев
- `ModelSettings` - настройки модели

Создадим модель, скомпилируем ее и обучим на обучающей выборке. После обучения оценим точность модели на тестовой выборке. Если точность больше 0.8, то сохраним результаты в файл `data/result_submission.csv`.

В качестве модели возьмем CNN с одним сверточным слоем, одним слоем пулинга, слоем дропаута, слоем выравнивания, двумя полносвязными слоями и выходным слоем.

In [73]:
class ImgSettings:
    img_rows = 28
    img_cols = 28
    
    @classmethod
    def get_img_shape(cls) -> tuple[int, int, int]:
        return cls.img_rows, cls.img_cols, 1
    
    @classmethod
    def reshape(cls, x: np.ndarray) -> np.ndarray:
        return x.reshape(x.shape[0], *cls.get_img_shape())


class LayersSettings:
    filters = 32
    kernel_size = 3
    activation = 'relu'
    pool_size = 2
    dropout = 0.2
    dense = 32
    dense_activation = 'relu'
    output = 10
    output_activation = 'softmax'


class ModelSettings:
    number_epochs = 17
    verbose = 1
    batch_size = 512
    loss = 'sparse_categorical_crossentropy'
    metrics = ['accuracy']
    
    write_graph = True
    histogram_freq = 1
    write_images = True


x_train = ImgSettings.reshape(x_train)
x_test = ImgSettings.reshape(x_test)
x_validate = ImgSettings.reshape(x_validate)

logger.info(f"Размер обучающей выборки: {x_train.shape}")
logger.info(f"Размер тестовой выборки: {x_test.shape}")
logger.info(f"Размер валидационной выборки: {x_validate.shape}")


model_layers = (
    layers.Input(shape=ImgSettings.get_img_shape()),
    layers.Conv2D(
        filters=LayersSettings.filters,
        kernel_size=LayersSettings.kernel_size, 
        activation=LayersSettings.activation,
    ),
    layers.MaxPooling2D(pool_size=LayersSettings.pool_size),
    layers.Dropout(LayersSettings.dropout),
    layers.Flatten(),
    layers.Dense(LayersSettings.dense, activation=LayersSettings.dense_activation),
    layers.Dense(LayersSettings.output, activation=LayersSettings.output_activation)
)

model = keras.Sequential(model_layers)
logger.info(f"Модель создана {model}")

tensorboard = callbacks.TensorBoard(
    write_graph=ModelSettings.write_graph,
    histogram_freq=ModelSettings.histogram_freq,
    write_images=ModelSettings.write_images,
)
logger.info(f"Tensorboard создан {tensorboard}")

model.compile(
    loss=ModelSettings.loss,
    metrics=ModelSettings.metrics,
)
logger.info(f"Модель скомпилирована {model}")

model.fit(
    x_train, 
    y_train, 
    batch_size=ModelSettings.batch_size,
    epochs=ModelSettings.number_epochs, 
    verbose=ModelSettings.verbose,
    validation_data=(x_validate, y_validate),
    callbacks=[tensorboard]
)

_, accuracy, *_ = model.evaluate(x_test, y_test, verbose=ModelSettings.verbose)

if accuracy > 0.8:
    logger.success(f"Точность модели: {accuracy}")
    
    y_response = model(x_test)
    
    logger.info("Начинаем сохранять результаты в файл")
    result = pd.DataFrame(np.argmax(y_response, axis=1), columns=["Category"])
    result["Id"] = result.index

    result.to_csv("data/result_submission.csv", index=False)
    logger.success("Результаты сохранены в файл")
else:
    logger.warning(f"Точность модели: {accuracy}. Нужно исправить модель 🥹")


2024-05-11 17:09:11.022 | INFO     | __main__:<module>:42 - Размер обучающей выборки: (48000, 28, 28, 1)
2024-05-11 17:09:11.027 | INFO     | __main__:<module>:43 - Размер тестовой выборки: (10000, 28, 28, 1)
2024-05-11 17:09:11.028 | INFO     | __main__:<module>:44 - Размер валидационной выборки: (12000, 28, 28, 1)
2024-05-11 17:09:11.070 | INFO     | __main__:<module>:62 - Модель создана <Sequential name=sequential_8, built=True>
2024-05-11 17:09:11.072 | INFO     | __main__:<module>:69 - Tensorboard создан <keras.src.callbacks.tensorboard.TensorBoard object at 0x3b831b450>
2024-05-11 17:09:11.077 | INFO     | __main__:<module>:75 - Модель скомпилирована <Sequential name=sequential_8, built=True>


Epoch 1/17
94/94 ━━━━━━━━━━━━━━━━━━━━ 6s 58ms/step - accuracy: 0.5765 - loss: 1.2389 - val_accuracy: 0.8082 - val_loss: 0.5447
Epoch 2/17
94/94 ━━━━━━━━━━━━━━━━━━━━ 6s 63ms/step - accuracy: 0.7979 - loss: 0.5525 - val_accuracy: 0.8428 - val_loss: 0.4537
Epoch 3/17
94/94 ━━━━━━━━━━━━━━━━━━━━ 3s 35ms/step - accuracy: 0.8357 - loss: 0.4573 - val_accuracy: 0.8438 - val_loss: 0.4358
Epoch 4/17
94/94 ━━━━━━━━━━━━━━━━━━━━ 3s 32ms/step - accuracy: 0.8557 - loss: 0.4063 - val_accuracy: 0.8478 - val_loss: 0.4166
Epoch 5/17
94/94 ━━━━━━━━━━━━━━━━━━━━ 4s 37ms/step - accuracy: 0.8679 - loss: 0.3695 - val_accuracy: 0.8657 - val_loss: 0.3721
Epoch 6/17
94/94 ━━━━━━━━━━━━━━━━━━━━ 3s 37ms/step - accuracy: 0.8769 - loss: 0.3452 - val_accuracy: 0.8793 - val_loss: 0.3375
Epoch 7/17
94/94 ━━━━━━━━━━━━━━━━━━━━ 3s 32ms/step - accuracy: 0.8869 - loss: 0.3256 - val_accuracy: 0.8824 - val_loss: 0.3276
Epoch 8/17
94/94 ━━━━━━━━━━━━━━━━━━━━ 3s 37ms/step - accuracy: 0.8907 - loss: 0.3111 - val_accuracy: 0.8820 - v

2024-05-11 17:10:10.278 | SUCCESS  | __main__:<module>:90 - Точность модели: 0.9064000248908997
2024-05-11 17:10:11.204 | INFO     | __main__:<module>:94 - Начинаем сохранять результаты в файл
2024-05-11 17:10:11.220 | SUCCESS  | __main__:<module>:99 - Результаты сохранены в файл
